# sprint 12 　　　ディープラーニング　フレームワーク  

In [67]:
from keras import backend as K

## 【問題1】スクラッチを振り返る  
１．重みを初期化する。  
２．学習モデルの決定。  
３．フォワード処理、バックワード処理、勾配降下法などで、より適したｗやｂを導出・更新。  
４．上記の２と３の処理を繰り返す（イタレーション、エポック）。  

## 【問題2】スクラッチとTensorFlowの対応を考える

In [20]:
# import urllib.request as req
# import pandas as pd

# # ファイルをダウンロードする
# url = "https://raw.githubusercontent.com/pandas-dev/pandas/master/pandas/tests/data/iris.csv" # 先ほどのURLではない
# savefile = "Iris.csv"
# req.urlretrieve(url, savefile)

# # ダウンロードしたファイルの内容を表示する
# csv = pd.read_csv(savefile, encoding="utf-8")
# csv

In [54]:
K.clear_session()

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Name"] == "Iris-versicolor")|(df["Name"] == "Iris-virginica")]
y = df["Name"]
X = df.loc[:, ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100

####　X placeholderに入力するデータ次元数 ４  ####
####　X placeholderに入力するデータサンプル数　１００   ####
####　Y placeholderに入力するクラス次元数 １   ####
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.1672, val_loss : 0.7845, acc : 1.000, val_acc : 0.938
Epoch 1, loss : 18.3028, val_loss : 46.0440, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 2.2667, val_loss : 3.2956, acc : 0.750, val_acc : 0.812
Epoch 3, loss : 0.0000, val_loss : 0.1466, acc : 1.000, val_acc : 0.938
Epoch 4, loss : 0.0000, val_loss : 0.4751, acc : 1.000, val_acc : 0.938
Epoch 5, loss : 0.0000, val_loss : 0.8386, acc : 1.000, val_acc : 0.938
Epoch 6, loss : 0.0000, val_loss : 0.5987, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 0.0022, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0000, val_loss : 0.2404, acc : 1.000, val_acc : 0.938
Epoch 9, loss : 0.0000, val_loss : 0.2433, acc : 1.000, val_acc : 0.938
test_acc : 0.900


・スクラッチにおけるＦｉｔ関数と、ＴＦにおけるネットワーク構造の読み込み（example_net）が対応していそう。  
・ＴＦでは、バックプロパゲーションの記述がない。  
・ＴＦでは、初期化がどのようにされているか不明。    
・ＴＦでは、最適化の際に、目的関数をminimizeするという指針がある（ネットワーク構造の最後の活性化関数が、最適化処理と同時になる）。  
・スクラッチではaccをsklearnで計算。ＴＦでは、accを自力で計算。

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [55]:
K.clear_session()

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
##############  データフレームから条件抽出  ############## 
df = df[(df["Name"] == "Iris-setosa")|(df["Name"] == "Iris-versicolor")|(df["Name"] == "Iris-virginica")]
y = df["Name"]
X = df.loc[:, ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]]
y = np.array(y)
X = np.array(X)
##############      ラベルを数値変換        ############## 
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

# y = y.astype(np.int)[:, np.newaxis]
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:,np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [53]:
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 12
num_epochs = 8
n_hidden1 = 50
n_hidden2 = 100

###########　Y placeholderに入力するクラス次元数 3   ##########
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None,n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み
logits = example_net(X)
#############       目的関数        ################
#loss_op = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y,logits = logits)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits = logits))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
###############       推定結果       ###############
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            # print(sess.run(temp, feed_dict={X: mini_batch_x, Y: mini_batch_y}))
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))

    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 11.1998, val_loss : 28.1083, acc : 0.500, val_acc : 0.583
Epoch 1, loss : 17.9112, val_loss : 17.4121, acc : 0.750, val_acc : 0.583
Epoch 2, loss : 4.5029, val_loss : 17.3537, acc : 0.750, val_acc : 0.667
Epoch 3, loss : 1.5907, val_loss : 10.8115, acc : 0.833, val_acc : 0.667
Epoch 4, loss : 6.2686, val_loss : 6.0041, acc : 0.833, val_acc : 0.792
Epoch 5, loss : 0.3766, val_loss : 11.7053, acc : 0.917, val_acc : 0.708
Epoch 6, loss : 1.4234, val_loss : 4.0520, acc : 0.917, val_acc : 0.833
Epoch 7, loss : 1.0658, val_loss : 3.2898, acc : 0.917, val_acc : 0.833
test_acc : 0.933


## 【問題5】MNISTのモデルを作成  　(問題４は後述)


In [56]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [58]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

In [60]:
X_train.shape
X_test.shape
y_test_one_hot.shape

(10000, 10)

In [62]:
K.clear_session()

class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

learning_rate = 0.025
batch_size = 20
num_epochs = 20
n_hidden1 = 400   ### 以前のNNと同様に設定
n_hidden2 = 200   ### 以前のNNと同様に設定

n_input = X_train.shape[1]   ### 784
n_samples = X_train.shape[0]   ### X_trainなら48000
n_classes = 10    ### クラス数１０

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None,n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.tanh(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.tanh(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

logits = example_net(X)
#############       目的関数        ################
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits = logits))

#最適化手法
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
###############       推定結果       ###############
correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            # print(sess.run(temp, feed_dict={X: mini_batch_x, Y: mini_batch_y}))
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))

    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.3889, val_loss : 1.9250, acc : 0.900, val_acc : 0.781
Epoch 1, loss : 0.2876, val_loss : 1.5650, acc : 0.950, val_acc : 0.814
Epoch 2, loss : 0.1109, val_loss : 1.4275, acc : 0.950, val_acc : 0.827
Epoch 3, loss : 0.0274, val_loss : 1.3493, acc : 1.000, val_acc : 0.832
Epoch 4, loss : 0.0292, val_loss : 1.3142, acc : 1.000, val_acc : 0.835
Epoch 5, loss : 0.0323, val_loss : 1.2770, acc : 1.000, val_acc : 0.836
Epoch 6, loss : 0.0345, val_loss : 1.2564, acc : 1.000, val_acc : 0.837
Epoch 7, loss : 0.0362, val_loss : 1.2385, acc : 1.000, val_acc : 0.838
Epoch 8, loss : 0.0390, val_loss : 1.2239, acc : 1.000, val_acc : 0.839
Epoch 9, loss : 0.0410, val_loss : 1.2118, acc : 1.000, val_acc : 0.840
Epoch 10, loss : 0.0418, val_loss : 1.2023, acc : 1.000, val_acc : 0.841
Epoch 11, loss : 0.0417, val_loss : 1.1922, acc : 1.000, val_acc : 0.844
Epoch 12, loss : 0.0426, val_loss : 1.1833, acc : 1.000, val_acc : 0.844
Epoch 13, loss : 0.0422, val_loss : 1.1746, acc : 1.000, val_

## 【問題4】House Pricesのモデルを作成

In [63]:
import pandas as pd
import IPython
import numpy as np

aims_data = pd.read_csv("train.csv")
aims_data4 = aims_data.loc[:,['GrLivArea','YearBuilt','SalePrice']]

In [64]:
from sklearn.model_selection import train_test_split
X = aims_data4.iloc[:,0:2].values
y = aims_data4.iloc[:,2].values
X = np.log(X)
y = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state = 1)
y_train=y_train[:,np.newaxis]
y_test=y_test[:,np.newaxis]
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state = 1)


In [65]:
print(y_train.shape, y_val.shape, y_test.shape)

(876, 1) (292, 1) (292, 1)


In [66]:
K.clear_session()

#####  tensorboard のための記憶フォルダ作成　#####
log_dir = 'our_graph'
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 10
n_hidden1 = 400
n_hidden2 = 100
n_hidden3 = 50

###########　Y placeholderに入力するクラス次元数 3   ##########
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None,n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_hidden3])),
        'w4': tf.Variable(tf.random_normal([n_hidden3, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_hidden3])),
        'b4': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_output = tf.matmul(layer_3, weights['w4']) + biases['b4'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み
logits = example_net(X)
#############       目的関数        ################
#loss_op = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y,logits = logits)
loss_op = tf.reduce_mean(tf.square(logits - Y))

#最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# ###############       推定結果       ###############
# #correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# correct_pred = logits

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss))

    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_loss : {:.3f}".format(test_loss))
    
    #####  tensorboard のためのファイル作成　#####
    summary_writer = tf.summary.FileWriter(log_dir , sess.graph)
    #####　anacondaプロンプトで、tensorboard --logdir=/tmp/・・・/our_graph
    #####  http://localhost:6006/ を開く

Epoch 0, loss : 11727.5430, val_loss : 9706.5908
Epoch 1, loss : 68.4588, val_loss : 110.0960
Epoch 2, loss : 10.9124, val_loss : 10.3980
Epoch 3, loss : 4.0550, val_loss : 3.9975
Epoch 4, loss : 12.2360, val_loss : 17.7899
Epoch 5, loss : 11.1419, val_loss : 16.3145
Epoch 6, loss : 28.7590, val_loss : 35.3295
Epoch 7, loss : 15.6917, val_loss : 20.9770
Epoch 8, loss : 21.9022, val_loss : 26.5628
Epoch 9, loss : 2.0190, val_loss : 3.9053
test_loss : 3.485
